In [ ]:
# 함수형 api를 활용한 모델 구성

In [1]:
from tensorflow.keras.datasets import mnist, cifar10
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
x_train_rs = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test_rs = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255

In [5]:
y_train_ohe=to_categorical(y_train)
y_test_ohe=to_categorical(y_test)

In [6]:
#함수형 API

In [13]:
inputs= Input(shape=(28,28,1))
x= Conv2D(32,(3,3),activation='relu')(inputs)
x= Conv2D(64,(3,3),activation='relu')(x)
x= MaxPooling2D((2,2))(x)
x= Dropout(0.25)(x)
x= Flatten()(x)
x= Dense(256,activation='relu')(x)
x= Dropout(0.25)(x)
outputs = Dense(10,activation='softmax')(x)

model=Model(inputs=inputs,outputs=x)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     2,359,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,378,368 (9.07 MB)

 Trainable params: 2,378,368 (9.07 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(x_train_rs, y_train_ohe,
          batch_size=32,
          validation_data=(x_test_rs, y_test_ohe),
          epochs=10)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(32, 10), output.shape=(32, 256)